In [6]:
import os,glob
import pandas as pd
import nibabel as nib
DATA_PATH="datasets/Zhongshan/pet-mr"
TAR_PATH="datasets/Zhongshan_prep"
CSV_PATH="datasets/Zhongshan/Diagnosis Information.csv"
%cd ..
%cd AD_GAN

c:\Users\CH2\Projects
c:\Users\CH2\Projects\AD_GAN


In [7]:
df=pd.read_csv(CSV_PATH,dtype=str,keep_default_na=False)
# df.set_index("PID")
pdict={value:key for key,value in df[df["diagonsis"]!=""]["PID"].to_dict().items()}
plist=[value for key,value in df[df["diagonsis"]!=""]["PID"].to_dict().items()]
plist

['ZS13339215',
 'ZS12344318',
 'ZS20108556',
 'ZS20270289',
 'ZS20270288',
 'ZS09005057',
 'ZS09176714',
 'ZS09069673',
 'ZS08002918',
 'ZS20276801',
 'ZS20276793',
 'ZS20276785',
 'ZS11344100',
 'ZS20276854',
 'ZS20276850',
 'ZS20276845',
 'ZS17142399',
 'ZS17281196',
 'ZS16268152',
 'ZS10312675',
 'ZS13229156',
 'ZS15049778',
 'ZS11070647',
 'ZS09037906',
 'ZS14049399',
 'ZS20283449',
 'ZS20283451',
 'ZS20283452',
 'ZS20283454',
 'ZS15149810',
 'ZS20283719',
 'ZS20283720',
 'ZS09067464',
 'ZS09128145',
 'ZS09004601',
 'ZS13004626',
 'ZS09024393',
 'ZS21007410',
 'ZS16043037',
 'ZS15168778',
 'ZS19075135',
 'ZS12151383',
 'ZS18247502',
 'ZS18020720',
 'ZS16319770',
 'ZS10015525',
 'ZS09150731',
 'ZS14103875',
 'ZS14249195',
 'ZS13090589',
 'ZS13069533',
 'ZS09230831',
 'ZS21056384',
 'ZS18360533',
 'ZS15224481',
 'ZS20072011',
 'ZS17244035',
 'ZS20026768',
 'ZS21064397',
 'ZS14179826',
 'ZS20201420',
 'ZS18192544',
 'ZS21064404',
 'ZS21086044',
 'ZS13037646',
 'ZS21086057',
 'ZS210860

In [18]:
import subprocess

def show(s,output=None):
    print(s,flush=True)
    # print(s,file=sys.stderr,flush=True)
    if output is not None:
        output.append(s+'\n')

def run_sh(cmd,name="unknown",base_dir="tmp",pname="unknown",outputs=[]):
    # tcmd=
    outputs.append(cmd)
    print(cmd)
    ret=subprocess.run(f"cd {base_dir} && {cmd} 2>&1",shell=True,stdout=subprocess.PIPE,encoding="utf")
    if ret.stdout is not None:
        outputs.append(ret.stdout+'\n')
    if ret.returncode!=0:
        show(f"{pname}.{name}: {cmd.split()[0]} Failed!",outputs)
        raise Exception(f"{name} Error!")
    show(f"{pname}.{name}: {cmd.split()[0]} Successfully!",outputs)


In [31]:
### 1. 拷贝已有图像
### ${TARGET_DIR}/${MOD}/${PID}/img1,img2...
# patient_dict={}
# for dirpath, dirnames, filenames in os.walk("."):
#     pid=os.path.basename(dirpath).split("_")
#     for poss_h in pid:
#         if poss_h in pdict:
#             df_id=pdict[poss_h]
#             diag=df.iloc()[df_id]["diagonsis"]
#             if diag not in patient_dict:patient_dict[diag]=[]
#             patient_dict[diag].append([poss_h,dirpath])

import glob
import shutil
def is_imgdir(dirpath):
    for mName in os.listdir(dirpath):
        if mName.startswith("t1_gre_fsp"):
            return True
    return False

def cpimg(dirpath:str,need_mod:list):
    try:
        mpid=""
        for pid in plist:
            if dirpath.find(pid)!=-1:
                mpid=pid
        if mpid=="":
            raise(Exception(f"pid not found in {dirpath}!"))

        for md,gr in need_mod:
            ptg=os.path.join(TAR_PATH,md,"05ZS",mpid)
            os.makedirs(ptg,exist_ok=True)

            for img in glob.glob(os.path.join(dirpath,gr)):
                # print(ptg,img)
                # if img.find("NIF")!=-1:

                ### TODO: useless in py 3.7

                # shutil.copytree(src=img,dst=ptg,dirs_exist_ok=True)
                run_sh(f"cp -r -t \"{ptg}\" \"{img}\"",base_dir=".",pname="cp {img}")
                # if img.endswith("zip"):
                    # run_sh(f"unzip {ptg}/{img}",base_dir=ptg,pname="cp {img}")
                # print(ptg,img)
        return True
    except Exception as e:
        print(e)
        return False

def get_img(need_mod:list):
    succ_cnt,fail_cnt=0,0
    fail_list=[]
    for dirpath, dirnames, filenames in os.walk(DATA_PATH):
        if is_imgdir(dirpath):
            if cpimg(dirpath,need_mod):
                succ_cnt+=1
            else:
                fail_cnt+=1
                fail_list.append(dirpath)
    print("Succ_cnt: ",succ_cnt)
    print("Fail_cnt: ",fail_cnt)
    print("Fail_list: ",fail_list)

need_mod=[("t1","t1_gre_fsp_3d_sag_0.8mm_[0-9]*"),("t2","t2_mx3d_sag_0.8mm_[0-9]*")]
get_img(need_mod)

[Errno 13] Permission denied: 'datasets/Zhongshan/pet-mr\\20201107\\Yang, Liping_ZS13339215_091308\\t1_gre_fsp_3d_sag_0.8mm_302'
Succ_cnt:  0
Fail_cnt:  1
Fail_list:  ['datasets/Zhongshan/pet-mr\\20201107\\Yang, Liping_ZS13339215_091308']


In [ ]:
### 2. 预处理T1，T2

In [ ]:
### 3. 配准，获得flt数据

In [ ]:
def load_t1_img(srcpath,tgpath):
    os.makedirs(tgpath,exist_ok=True)
    sh=lambda cmd,name="unknown",base_dir=".":run_sh(cmd,name=name,base_dir=base_dir,pname=srcpath,outputs=[])
    srcpath=srcpath+"/Image/t1_gre_fsp_3d_sag_0.8mm_[0-9]*"
    srcpath=sorted(glob.glob(srcpath))[0]
    sh(f"cp -t {tgpath} {srcpath}/*",name="cp",base_dir=srcpath)
    sh(f"dcm2niix -b y -z y -x n -t n -m n -f t1 -o . -s n -v n .",name="dcm2nii",base_dir=tgpath)
    img=nib.load("t1.nii.gz")
    img_arr=img.get_fdata()
    return img_arr

In [ ]:
os.makedirs(TAR_PATH,exist_ok=True)
NC_PATH,MCI_PATH=TAR_PATH+"/NC",TAR_PATH+"/MCI"
os.makedirs(NC_PATH,exist_ok=True)
os.makedirs(MCI_PATH,exist_ok=True)
data=[]
for pid,dirpath in patient_dict["NC"]:
    data.append([load_t1_img(dirpath,NC_PATH+"/"+pid),0])
for pid,dirpath in patient_dict["MCI"]:
    data.append([load_t1_img(dirpath,MCI_PATH+"/"+pid),1])

In [54]:
df.iloc()[49]["diagonsis"]

''